# Word Embededing based Text classification 

## by Yash Tripathi


In this document we will be following the Text classification method using Word embededing and DNN Models to classify the data and get the Accuracy Result.
It will be then compared with the other Predictions done by my other team memnbers and then we will choose the most aplicable model from the set.

Team Members:-
[@Yash Tripahti](https://drkakku.github.io/GoV2/todo.html)
[@Hitesh Goyal](https://drkakku.github.io/GoV2/todo.html)
[@Ashwin Iyer](https://drkakku.github.io/GoV2/todo.html)
[@Rishma ](https://drkakku.github.io/GoV2/todo.html)
[@Rahul Dash](https://drkakku.github.io/GoV2/todo.html)





The basic Pipeline that i will be following in this case will be :- 

- Split the data into text (X) and labels (Y)
- Preprocess X
- Create a word embedding matrix from X
- Create a tensor input from X
- Train a deep learning model using the tensor inputs and labels (Y)
- Make predictions on new data


In [9]:
import pandas as pd
import numpy as np
import seaborn as sea


In [10]:
trainDf = pd.read_csv("../input/train.csv",index_col=0)
testDf = pd.read_csv("../input/test.csv",index_col=0)
validationDf = pd.read_csv("../input/validation.csv",index_col=0)


In [11]:
trainDf.head()

,title,text,label
6457,ashton kutcher rescues sex trafficking victims...,christopher ashton kutcher wellknown figure am...,FAKE
52107,florida crowd attacks police officer attemptin...,citizenry respect law law enforcement officers...,FAKE
55294,oracle profit rises software demand,san francisco reuters oracle corp tuesday repo...,REAL
27913,erdogan pope say phone call attempts change je...,ankara reuters turkish president tayyip erdoga...,REAL
59719,allrounder mcgrath,glenn mcgrath thoroughbred fast bowler decade ...,REAL


Now we will do some basic NLP preprosessing of the text 

In [12]:
import re


In [13]:
def cleanText(string:str ,
    punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
    stop_words=['the', 'a', 'and', 'is', 'be', 'will']) -> str:

    #Cleaning URL's
    string = re.sub(r'https?://\S+|www\.\S+',"",string)

    #cleaning the HTML elements
    string = re.sub(r'<.*?>',"",string)

    # removing the puntuations

    for x in string.lower():
        if x in punctuations:
            string = string.replace(x,"")

    #converting text to lower

    string  = string.lower()

    # removing the stop words
    string = " ".join([word for word in string.split() if word not in stop_words])

    #Cleaning the white space
    string = re.sub(r'\s+'," ",string=string).strip()

    return string




In [14]:
ValDfX , ValDfY = validationDf["title"] , validationDf["label"]


In [17]:
import numpy as np


class Embeddings():
    """
    A class to read the word embedding file and to create the word embedding matrix
    """

    def __init__(self, path, vector_dimension):
        self.path = path 
        self.vector_dimension = vector_dimension
    
    @staticmethod
    def get_coefs(word, *arr): 
        return word, np.asarray(arr, dtype='float32')

    def get_embedding_index(self):
        embeddings_index = dict(self.get_coefs(*o.split(" ")) for o in open(self.path, errors='ignore'))
        return embeddings_index

    def create_embedding_matrix(self, tokenizer, max_features):
        """
        A method to create the embedding matrix
        """
        model_embed = self.get_embedding_index()

        embedding_matrix = np.zeros((max_features + 1, self.vector_dimension))
        for word, index in tokenizer.word_index.items():
            if index > max_features:
                break
            else:
                try:
                    embedding_matrix[index] = model_embed[word]
                except:
                    continue
        return embedding_matrix

In [22]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
     

class TextToTensor():


    def __init__(self, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len

    def string_to_tensor(self, string_list: list) -> list:
        """
        A method to convert a string list to a tensor for a deep learning model
        """    
        string_list = self.tokenizer.texts_to_sequences(string_list)
        string_list = pad_sequences(string_list, maxlen=self.max_len)
        
        return string_list

In [23]:
from keras.preprocessing.text import Tokenizer
# Tokenizing the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(ValDfX)
# Getting the longest sentence
max_len = np.max([len(text.split()) for text in ValDfX])
# Converting to tensor
TextToTensor_instance = TextToTensor(
tokenizer=tokenizer,
max_len=max_len
)
X_train_NN = TextToTensor_instance.string_to_tensor(ValDfX)

In [29]:
embed_path = '../input/embedings/glove.6B.300d.txt'
embed_dim = 300
# Tokenizing the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(ValDfX)
# Creating the embedding matrix
embedding = Embeddings(embed_path, embed_dim)
embedding_matrix = embedding.create_embedding_matrix(tokenizer, len(tokenizer.word_counts))

In [34]:
np.shape(X_train_NN)

(10699, 42)

In [36]:
from keras.models import Sequential
from keras.layers import Embedding
# Converting to tensor
TextToTensor_instance = TextToTensor(
tokenizer=tokenizer,
max_len=max_len
)
X_train_NN = TextToTensor_instance.string_to_tensor(ValDfX)
model = Sequential()
model.add(Embedding(
  input_dim=14742, 
  output_dim=300, 
  input_length=max_len,
  weights=[embedding_matrix]))

model.compile('rmsprop', 'mse')
output_array = model.predict(X_train_NN)[0]

(42, 300)